# Block-Silhouette Analysis for CogSci 2020

## Import and set up directories

In [ ]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
proj_dir = os.path.abspath('../..')

if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))

import numpy as np
import scipy.stats as stats
import pandas as pd
from random import random

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

import blockworld_helpers as utils
import drawing_utils as drawing
import importlib

In [ ]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.path.abspath(os.path.join(os.getcwd(),'..'))
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))
jefan_dir = os.path.join(analysis_dir,'jefan')
will_dir = os.path.join(analysis_dir,'will')

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

## Load Data

In [ ]:
iteration_name = 'Exp2Pilot2'
num_trials = 24 #for sanity checks

In [ ]:
# Data already compiled into dataframes in CogSci 2020 Dataframe Generator

# Trial_end data
trial_path = os.path.join(csv_dir,'block_silhouette_{}.csv'.format(iteration_name))
df = pd.read_csv(trial_path)

# # Settled_block data
# settled_path = os.path.join(csv_dir,'block_silhouette_settled_{}.csv'.format(iterationName))
# dfs = pd.read_csv(settled_path)

# # Sanity Check- same participants in each dataset.
# df_participants = df.gameID.unique()
# dfs_participants = dfs.gameID.unique()
# assert collections.Counter(df_participants) == collections.Counter(dfs_participants)

# n_before_outliers = len(df_participants)
# print(str(n_before_outliers) + ' participants total')

In [ ]:
# Exclude outliers
low_scores = df[(df.trialNum == (num_trials-1)) & (df.score == 0)]['gameID'] # people who achieved zero bonus

df = df[~df.gameID.isin(low_scores)]
#dfs = dfs[~dfs.gameID.isin(low_scores)]

n = df.gameID.nunique()
print(str(n) + ' participants with score > 0 and full set of trials')


In [ ]:
importlib.reload(drawing)
drawing.draw_all_trials(df, figsize=(10,80))

In [ ]:
d1 = df.groupby(['gameID','condition','phase'])['normedScoreDiscrete'].apply(np.mean)
d1 = d1.reset_index()
d1

In [ ]:
fig = plt.figure(figsize=(8,6))
sns.set_context('poster')
sns.set_style('whitegrid', {'legend.frameon':False})
c = sns.catplot(x='repetition', 
                y='normedScoreDiscrete',
                hue='condition',
                kind='point', 
                legend=d1,
                data=df);
plt.ylabel('Accuracy')
plt.xlabel('Repetition')
#plt.yticks(np.linspace(0.4,1,5))
plt.setp(c.ax.lines,linewidth=2)
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(8,6))
sns.set_context('poster')
sns.set_style('whitegrid', {'legend.frameon':False})
c = sns.barplot(x='repetition', 
                y='normedScoreDiscrete', 
                hue='condition', 
                #kind='point', 
                #legend=False,
                data=df)
plt.ylabel('Accuracy')
plt.xlabel('Repetition')
plt.title('Accuracy by repetition (control repetition 1 happens at same time as repeated repetition 3)\n',{'fontsize': 16})
plt.ylim(0.4,1)
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)

In [ ]:
fig = plt.figure(figsize=(8,6))
sns.set_context('poster')
sns.set_style('whitegrid', {'legend.frameon':False})
c = sns.barplot(x='repetition', 
                y='timeToBuild', 
                hue='condition', 
                #kind='point', 
                #legend=False,
                data=df)
plt.ylabel('time to build (ms)')
plt.xlabel('Repetition')
plt.title('Build time by repetition (control repetition 1 happens at same time as repeated repetition 3)\n',{'fontsize': 16})
#plt.ylim(0.4,1)
plt.tight_layout()
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)

In [ ]:
fig = plt.figure(figsize=(8,6))
sns.set_context('poster')
sns.set_style('whitegrid', {'legend.frameon':False})
c = sns.barplot(x='phase', 
                y='numBlocks', 
                hue='condition', 
                #kind='point', 
                #legend=False,
                data=df);
plt.ylabel('Number Blocks Used')
plt.xlabel('Phase')
#plt.yticks(np.linspace(0.4,1,5))
plt.tight_layout()

fig = plt.figure(figsize=(8,6))
sns.set_context('poster')
sns.set_style('whitegrid', {'legend.frameon':False})
c = sns.barplot(x='phase', 
                y='normedScoreDiscrete', 
                hue='condition', 
                #kind='point', 
                #legend=False,
                data=df);
plt.ylabel('Normed F1 Score')
plt.xlabel('Phase')
plt.ylim(0.4,1)
plt.tight_layout()

In [ ]:
subject_means_by_condition = df.groupby(['gameID','condition']).mean()
condition_means = subject_means_by_condition.groupby('condition')['normedScore'].mean()

subject_scores_by_condition = subject_means_by_condition['normedScore']

fig = plt.figure(figsize=(8,6))
sns.set_context('poster')
sns.set_style('whitegrid', {'legend.frameon':False})
c = sns.catplot(x='condition', 
                y='normedScore', 
                hue='gameID', 
                kind='point', 
                legend=False,
                data=df);
plt.ylabel('Normed F1 score')
plt.xlabel('Condition')
plt.yticks(np.linspace(0.4,1,5))
plt.setp(c.ax.lines,linewidth=2)
plt.tight_layout()